## Data

In [1]:
import torch
import torch.nn as nn
import torchtext; torchtext.disable_torchtext_deprecation_warning()
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

corpus = [
    "ăn quả nhớ kẻ trồng cây",
    "có chí thì nên"    
]
data_size = len(corpus)

# Define the max vocabulary size and sequence length
vocab_size = 13
sequence_length = 7

In [2]:
# Define tokenizer function
tokenizer = get_tokenizer('basic_english')

# Create a function to yield list of tokens
def yield_tokens(examples):
    for text in examples:
        yield tokenizer(text)

# Create vocabulary
vocab = build_vocab_from_iterator(yield_tokens(corpus),
                                  max_tokens=vocab_size,
                                  specials=["<unk>", "<pad>", "<sos>"]
                                 )
vocab.set_default_index(vocab["<unk>"])
vocab.get_stoi()

{'ăn': 12,
 'nên': 8,
 'nhớ': 7,
 'có': 5,
 'thì': 10,
 'trồng': 11,
 'kẻ': 6,
 '<unk>': 0,
 'cây': 4,
 'quả': 9,
 '<sos>': 2,
 'chí': 3,
 '<pad>': 1}

In [3]:
data_x = []
data_y = []
for vector in corpus:
    vector = vector.split()    

    for i in range(len(vector)):
        data_x.append(vector[:i])
        data_y.append(vector[i])

print(data_x)
print(data_y)

[[], ['ăn'], ['ăn', 'quả'], ['ăn', 'quả', 'nhớ'], ['ăn', 'quả', 'nhớ', 'kẻ'], ['ăn', 'quả', 'nhớ', 'kẻ', 'trồng'], [], ['có'], ['có', 'chí'], ['có', 'chí', 'thì']]
['ăn', 'quả', 'nhớ', 'kẻ', 'trồng', 'cây', 'có', 'chí', 'thì', 'nên']


In [4]:
# Tokenize and numericalize your samples
def vectorize(x, y, vocab, sequence_length):
    x = ['<sos>'] + x
    print(x)
    x_ids = [vocab[token] for token in x][:sequence_length]
    x_ids = x_ids + [vocab["<pad>"]] * (sequence_length - len(x))
    
    return x_ids, vocab[y]

# Vectorize the samples
data_x_ids = []
data_y_ids = []
for x, y in zip(data_x, data_y):
    x_ids, y_ids = vectorize(x, y, vocab, sequence_length)
    data_x_ids.append(x_ids)
    data_y_ids.append(y_ids)

['<sos>']
['<sos>', 'ăn']
['<sos>', 'ăn', 'quả']
['<sos>', 'ăn', 'quả', 'nhớ']
['<sos>', 'ăn', 'quả', 'nhớ', 'kẻ']
['<sos>', 'ăn', 'quả', 'nhớ', 'kẻ', 'trồng']
['<sos>']
['<sos>', 'có']
['<sos>', 'có', 'chí']
['<sos>', 'có', 'chí', 'thì']


In [5]:
for x, y in zip(data_x_ids, data_y_ids):
    print(x)
    print(y)
    print()

[2, 1, 1, 1, 1, 1, 1]
12

[2, 12, 1, 1, 1, 1, 1]
9

[2, 12, 9, 1, 1, 1, 1]
7

[2, 12, 9, 7, 1, 1, 1]
6

[2, 12, 9, 7, 6, 1, 1]
11

[2, 12, 9, 7, 6, 11, 1]
4

[2, 1, 1, 1, 1, 1, 1]
5

[2, 5, 1, 1, 1, 1, 1]
3

[2, 5, 3, 1, 1, 1, 1]
10

[2, 5, 3, 10, 1, 1, 1]
8



In [6]:
data_x_ids = torch.tensor(data_x_ids, dtype=torch.long)
print(data_x_ids.shape)

torch.Size([10, 7])


In [7]:
data_y_ids = torch.tensor(data_y_ids, dtype=torch.long)
print(data_y_ids.shape)

torch.Size([10])


## Model

In [8]:
class TG_Model(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_heads, sequence_length):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)        
        self.mask = torch.triu(torch.ones(sequence_length, sequence_length), 
                               diagonal=1).bool()
        self.transformer =  nn.TransformerEncoderLayer(d_model=embed_dim, 
                                                       nhead=num_heads, 
                                                       batch_first=True, 
                                                       dim_feedforward=4,
                                                       dropout=0.0)
        self.linear = nn.Linear(sequence_length*embed_dim, vocab_size)

    def forward(self, x):
        x = self.embedding(x)                         # [n, seq_len, embed_dim]
        x = self.transformer(x, src_mask=self.mask)   # [n, seq_len, embed_dim]
        x = nn.Flatten()(x)                           # [n, seq_len*embed_dim]
        x = self.linear(x)                            # [n, vocab_size]
        return x

model = TG_Model(vocab_size, 8, 2, sequence_length)
print(model)

TG_Model(
  (embedding): Embedding(13, 8)
  (transformer): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=8, out_features=8, bias=True)
    )
    (linear1): Linear(in_features=8, out_features=4, bias=True)
    (dropout): Dropout(p=0.0, inplace=False)
    (linear2): Linear(in_features=4, out_features=8, bias=True)
    (norm1): LayerNorm((8,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((8,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.0, inplace=False)
    (dropout2): Dropout(p=0.0, inplace=False)
  )
  (linear): Linear(in_features=56, out_features=13, bias=True)
)


In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.05)

## Train

In [10]:
# train
for _ in range(40):
    optimizer.zero_grad()
    outputs = model(data_x_ids)
    loss = criterion(outputs, data_y_ids)
    print(loss.item())
    loss.backward()
    optimizer.step()

2.8626513481140137
2.0063111782073975
1.2720909118652344
0.8711012601852417
0.5769220590591431
0.40428152680397034
0.2984568178653717
0.22718572616577148
0.18501654267311096
0.16782192885875702
0.15720482170581818
0.15095987915992737
0.14727486670017242
0.1463598757982254
0.14388756453990936
0.14234121143817902
0.14209994673728943
0.14094005525112152
0.14044535160064697
0.14074596762657166
0.1402960568666458
0.13945385813713074
0.13931569457054138
0.13970980048179626
0.13987641036510468
0.13957563042640686
0.13910405337810516
0.1388550102710724
0.1389150470495224
0.13919992744922638
0.13965818285942078
0.14026086032390594
0.1409301459789276
0.14117582142353058
0.14065007865428925
0.13950571417808533
0.1387500762939453
0.13891030848026276
0.1396038681268692
0.140054851770401


In [11]:
outputs = model(data_x_ids)
#print(outputs)
print(torch.argmax(outputs, axis=-1))

tensor([ 5,  9,  7,  6, 11,  4,  5,  3, 10,  8])


In [12]:
data_y_ids

tensor([12,  9,  7,  6, 11,  4,  5,  3, 10,  8])

## Inference

In [13]:
promt = '<sos> có'
promt = promt.split()
promt_ids = [vocab[token] for token in promt][:sequence_length]
promt_ids = promt_ids + [vocab["<pad>"]] * (sequence_length - len(promt))

print(promt_ids)

[2, 5, 1, 1, 1, 1, 1]


In [14]:
for i in range(sequence_length - len(promt)):
    promt_tensor = torch.tensor(promt_ids, dtype=torch.long).reshape(1, -1)
    outputs = model(promt_tensor)
    next_id = torch.argmax(outputs, axis=-1)

    promt_ids[len(promt)+i] = next_id.item()
    print(promt_ids)

[2, 5, 3, 1, 1, 1, 1]
[2, 5, 3, 10, 1, 1, 1]
[2, 5, 3, 10, 8, 1, 1]
[2, 5, 3, 10, 8, 8, 1]
[2, 5, 3, 10, 8, 8, 8]


In [ ]:
{'ăn': 12,
 'nên': 8,
 'nhớ': 7,
 'có': 5,
 'thì': 10,
 'trồng': 11,
 'kẻ': 6,
 '<unk>': 0,
 'cây': 4,
 'quả': 9,
 '<sos>': 2,
 'chí': 3,
 '<pad>': 1}